# Mistral मॉडेल्ससह बांधकाम

## परिचय

हा धडा यावर लक्ष केंद्रित करेल:
- वेगवेगळ्या Mistral मॉडेल्सचा शोध घेणे
- प्रत्येक मॉडेलसाठी वापर प्रकरणे आणि परिस्थिती समजून घेणे
- कोड नमुने प्रत्येक मॉडेलच्या अनन्य वैशिष्ट्ये दाखवतात.


## मिस्त्राल मॉडेल्स

या धड्यात, आपण 3 वेगवेगळ्या मिस्त्राल मॉडेल्सचा अभ्यास करू:  
**मिस्त्राल लार्ज**, **मिस्त्राल स्मॉल** आणि **मिस्त्राल नेमो**.

या प्रत्येक मॉडेल्स Github मॉडेल मार्केटप्लेसवर मोफत उपलब्ध आहेत. या नोटबुकमधील कोड या मॉडेल्सचा वापर करून चालवला जाईल. Github मॉडेल्स वापरून [AI मॉडेल्ससह प्रोटोटायपिंग](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) कसे करायचे याबद्दल अधिक माहिती येथे आहे.


## Mistral Large 2 (2407)
Mistral Large 2 सध्या Mistral कडून प्रमुख मॉडेल आहे आणि ते एंटरप्राइझ वापरासाठी डिझाइन केलेले आहे.

हे मॉडेल मूळ Mistral Large चे एक अपग्रेड आहे जे ऑफर करते
- मोठा संदर्भ विंडो - 128k विरुद्ध 32k
- गणित आणि कोडिंग कार्यांवर चांगले कामगिरी - 76.9% सरासरी अचूकता विरुद्ध 60.4%
- वाढलेली बहुभाषिक कामगिरी - भाषांमध्ये समाविष्ट आहेत: इंग्रजी, फ्रेंच, जर्मन, स्पॅनिश, इटालियन, पोर्तुगीज, डच, रशियन, चिनी, जपानी, कोरियन, अरबी, आणि हिंदी.

या वैशिष्ट्यांसह, Mistral Large मध्ये उत्कृष्टता आहे
- *Retrieval Augmented Generation (RAG)* - मोठ्या संदर्भ विंडोमुळे
- *Function Calling* - या मॉडेलमध्ये स्थानिक फंक्शन कॉलिंग आहे जे बाह्य साधने आणि API सह एकत्रीकरणाची परवानगी देते. हे कॉल्स एकाच वेळी किंवा अनुक्रमे एकानंतर एक केले जाऊ शकतात.
- *Code Generation* - हे मॉडेल Python, Java, TypeScript आणि C++ जनरेशनमध्ये उत्कृष्ट आहे.


### Mistral Large 2 वापरून RAG उदाहरण


या उदाहरणात, आम्ही Mistral Large 2 वापरून एका मजकूर दस्तऐवजावर RAG पॅटर्न चालवत आहोत. प्रश्न कोरियन भाषेत लिहिला आहे आणि लेखकाच्या कॉलेजपूर्वीच्या क्रियाकलापांबद्दल विचारतो.

हे Cohere Embeddings Model वापरून मजकूर दस्तऐवज तसेच प्रश्न यांचे एम्बेडिंग तयार करते. या नमुन्यासाठी, faiss Python पॅकेज व्हेक्टर स्टोअर म्हणून वापरले जाते.

Mistral मॉडेलला पाठवलेला प्रॉम्प्ट प्रश्न आणि प्रश्नाशी संबंधित मिळवलेल्या भागांचा समावेश करतो. नंतर मॉडेल नैसर्गिक भाषेतील उत्तर प्रदान करते.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small हा Mistral कुटुंबातील आणखी एक मॉडेल आहे जो प्रीमियर/एंटरप्राइझ श्रेणीत येतो. नावाप्रमाणे, हे मॉडेल एक Small Language Model (SLM) आहे. Mistral Small वापरण्याचे फायदे म्हणजे: 
- Mistral LLMs जसे की Mistral Large आणि NeMo यांच्या तुलनेत खर्च बचत - ८०% किंमत कपात
- कमी विलंब - Mistral च्या LLMs च्या तुलनेत जलद प्रतिसाद
- लवचिक - आवश्यक संसाधनांवर कमी निर्बंधांसह विविध वातावरणांमध्ये तैनात करता येते. 


Mistral Small उत्तम आहे: 
- मजकूर आधारित कार्यांसाठी जसे की सारांश तयार करणे, भावना विश्लेषण आणि भाषांतर. 
- ज्या अनुप्रयोगांमध्ये वारंवार विनंत्या केल्या जातात कारण ते खर्चिकदृष्ट्या प्रभावी आहे 
- कमी विलंब असलेली कोड कार्ये जसे की पुनरावलोकन आणि कोड सूचना 


## Mistral Small आणि Mistral Large यांची तुलना

Mistral Small आणि Large यांच्यातील विलंबातील फरक दाखवण्यासाठी, खालील सेल्स चालवा.

आपण 3-5 सेकंदांच्या दरम्यान प्रतिसाद वेळांमध्ये फरक पाहू शकता. तसेच त्याच प्रॉम्प्टवर प्रतिसादाची लांबी आणि शैली लक्षात घ्या.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

या धड्यात चर्चा केलेल्या इतर दोन मॉडेल्सच्या तुलनेत, Mistral NeMo हा एकमेव मोफत मॉडेल आहे ज्याला Apache2 परवाना आहे.

हे Mistral कडून आधीच्या मुक्त स्रोत LLM चा अपग्रेड म्हणून पाहिले जाते, Mistral 7B.

NeMo मॉडेलची काही इतर वैशिष्ट्ये आहेत:

- *अधिक कार्यक्षम टोकनायझेशन:* हे मॉडेल अधिक सामान्यपणे वापरल्या जाणाऱ्या tiktoken पेक्षा Tekken टोकनायझर वापरते. यामुळे अधिक भाषा आणि कोडवर चांगली कामगिरी होते.

- *फाइनट्यूनिंग:* बेस मॉडेल फाइनट्यूनिंगसाठी उपलब्ध आहे. यामुळे ज्या वापरप्रकरणांसाठी फाइनट्यूनिंग आवश्यक आहे त्यासाठी अधिक लवचिकता मिळते.

- *नेटिव्ह फंक्शन कॉलिंग* - Mistral Large प्रमाणे, या मॉडेलला फंक्शन कॉलिंगवर प्रशिक्षण दिले गेले आहे. त्यामुळे हे एकमेव पहिले मुक्त स्रोत मॉडेल्सपैकी एक आहे जे असे करते.


## Mistral NeMo

या धड्यात चर्चा केलेल्या इतर दोन मॉडेल्सच्या तुलनेत, Mistral NeMo हा एकमेव मोफत मॉडेल आहे ज्याला Apache2 परवाना आहे.

हे Mistral कडून आधीच्या मुक्त स्रोत LLM चा अपग्रेड म्हणून पाहिले जाते, Mistral 7B.

NeMo मॉडेलची काही इतर वैशिष्ट्ये:

- *अधिक कार्यक्षम टोकनायझेशन:* हे मॉडेल अधिक सामान्यपणे वापरल्या जाणाऱ्या tiktoken पेक्षा Tekken टोकनायझर वापरते. यामुळे अधिक भाषा आणि कोडवर चांगली कामगिरी होते.

- *फाइनट्यूनिंग:* बेस मॉडेल फाइनट्यूनिंगसाठी उपलब्ध आहे. यामुळे ज्या वापरप्रकरणांसाठी फाइनट्यूनिंग आवश्यक आहे त्यासाठी अधिक लवचिकता मिळते.

- *नेटिव्ह फंक्शन कॉलिंग* - Mistral Large प्रमाणे, या मॉडेलला फंक्शन कॉलिंगवर प्रशिक्षण दिले गेले आहे. त्यामुळे हे एकमेव पहिले मुक्त स्रोत मॉडेल्सपैकी एक आहे जे असे करते.


### टोकनायझर्सची तुलना

या नमुन्यात, आपण पाहणार आहोत की Mistral NeMo टोकनायझेशन कसे हाताळते Mistral Large च्या तुलनेत.

दोन्ही नमुने एकाच प्रॉम्प्ट घेतात पण तुम्हाला दिसेल की NeMo तुलनेत Mistral Large पेक्षा कमी टोकन्स परत करते.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## शिकणे येथे थांबत नाही, प्रवास सुरू ठेवा

हा धडा पूर्ण केल्यानंतर, आमच्या [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) ला भेट द्या आणि तुमचे Generative AI ज्ञान अधिक वाढवा!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:
हा दस्तऐवज AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून अनुवादित केला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित अनुवादांमध्ये चुका किंवा अचूकतेची कमतरता असू शकते. मूळ दस्तऐवज त्याच्या स्थानिक भाषेत अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी अनुवाद शिफारसीय आहे. या अनुवादाच्या वापरामुळे उद्भवलेल्या कोणत्याही गैरसमजुती किंवा चुकीच्या अर्थलागी आम्ही जबाबदार नाही.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
